In [1]:
import sys 
sys.path.append("../../")

from br_demography  import municipality_migration as mm
import basedosdados as bd
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np

# Carrega as variáveis de ambiente do arquivo .env
load_dotenv(dotenv_path='../../.env')

# Use as variáveis de ambiente conforme necessário
project_id = os.getenv('GOOGLE_CLOUD_PROJECT_ID')

##### AQUI NÃO É POSSÍVEL FAZER UMA AVALIAÇÃO DE QUANTO SERÁ O SALDO MIGRATÓRIO DA REGIÃO, POIS OS DADOS AGREGADOS PARA CADA MUNICÍPIO
##### OU SEJA, O SALDO MIGRATÓRIO DE UM MUNICÍPIO INCLUI OS TROCAS INCLUSIVE COM OUTROS MUNICÍPIOS DA RM, OU SEJA, QUE NAO ALTERAM A POPULAÇÃO REGIONAL

In [2]:
mm.query_total_population(mun_id=4125506, project_id=project_id)

Downloading: 100%|██████████| 1/1 [00:00<00:00,  2.39rows/s]


,f0_
0,264210.0


In [3]:
df_mun = pd.read_csv('../../br_demography/source/tab/cod_mun.csv', sep=';', index_col='mun_id')

dict_dfs = dict()

for mun_id, mun in df_mun.itertuples():
    df_immigration_mun_2000 = mm.query_immigration_by_sex_age_2000(mun_id=mun_id, project_id=project_id)
    df_immigration_mun_2000 = mm.standard_age_groups(df=df_immigration_mun_2000, age_group_csv_path='../../br_demography/source/tab/faixas_etarias_censo_2000_2010.csv')
    df_immigration_mun_2000['Município'] = mun
    df_immigration_mun_2000['Ano'] = 2000
    df_immigration_mun_2000 = df_immigration_mun_2000.reset_index().set_index(['Ano', 'Município', 'Sexo','Faixa Etária'])

    df_immigration_mun_2010 = mm.query_immigration_by_sex_age(mun_id=mun_id, project_id=project_id)
    df_immigration_mun_2010 = mm.standard_age_groups(df=df_immigration_mun_2010, age_group_csv_path='../../br_demography/source/tab/faixas_etarias_censo_2000_2010.csv')
    df_immigration_mun_2010['Município'] = mun
    df_immigration_mun_2010['Ano'] = 2010
    df_immigration_mun_2010 = df_immigration_mun_2010.reset_index().set_index(['Ano', 'Município', 'Sexo','Faixa Etária'])

    df_immigration_mun = pd.concat(objs=[df_immigration_mun_2000, df_immigration_mun_2010])

    dict_dfs[mun] = df_immigration_mun

df_immigrantion = pd.concat(objs=dict_dfs.values())
df_immigrantion

Downloading: 100%|██████████| 54/54 [00:00<00:00, 178.88rows/s]


Peso
Ano  Município      Sexo      Faixa Etária              
2000 Adrianópolis   Feminino  0 a 9 anos       10.293958
                              10 a 19 anos     66.919097
                              20 a 29 anos     25.462694
                              30 a 39 anos     31.770795
                              40 a 49 anos     12.823922
...                                                  ...
2010 Doutor Ulysses Masculino 40 a 49 anos     16.192537
                              50 a 59 anos      0.000000
                              60 a 69 anos      2.565268
                              70 a 79 anos      0.000000
                              80 anos ou mais   0.000000

[1044 rows x 1 columns]

In [4]:
dict_dfs = dict()

for mun_id, mun in df_mun.itertuples():

    df_emigration_mun_2000 = mm.query_emigration_by_sex_age_2000(mun_id=mun_id, project_id=project_id)
    df_emigration_mun_2000 = mm.standard_age_groups(df=df_emigration_mun_2000, age_group_csv_path='../../br_demography/source/tab/faixas_etarias_censo_2000_2010.csv')
    df_emigration_mun_2000['Município'] = mun
    df_emigration_mun_2000['Ano'] = 2000
    df_emigration_mun_2000 = df_emigration_mun_2000.reset_index().set_index(['Ano', 'Município', 'Sexo','Faixa Etária'])

    df_emigration_mun_2010 = mm.query_emigration_by_sex_age(mun_id=mun_id, project_id=project_id)
    df_emigration_mun_2010 = mm.standard_age_groups(df=df_emigration_mun_2010, age_group_csv_path='../../br_demography/source/tab/faixas_etarias_censo_2000_2010.csv')
    df_emigration_mun_2010['Município'] = mun
    df_emigration_mun_2010['Ano'] = 2010
    df_emigration_mun_2010 = df_emigration_mun_2010.reset_index().set_index(['Ano', 'Município', 'Sexo','Faixa Etária'])

    df_emigration_mun = pd.concat(objs=[df_emigration_mun_2000, df_emigration_mun_2010])

    dict_dfs[mun] = df_emigration_mun

df_emigrantion = pd.concat(objs=dict_dfs.values())
df_emigrantion

Downloading: 100%|██████████| 57/57 [00:00<00:00, 182.57rows/s]


Peso
Ano  Município      Sexo      Faixa Etária               
2000 Adrianópolis   Feminino  0 a 9 anos        55.555053
                              10 a 19 anos     131.291200
                              20 a 29 anos     182.137986
                              30 a 39 anos      62.451511
                              40 a 49 anos      48.380681
...                                                   ...
2010 Doutor Ulysses Masculino 40 a 49 anos      28.248611
                              50 a 59 anos       0.000000
                              60 a 69 anos      12.307835
                              70 a 79 anos       0.000000
                              80 anos ou mais    0.000000

[1044 rows x 1 columns]

## Calculando a Proporção de cada Faixa Etária no Saldo Migratório

In [26]:
df__proporcao_saldo = ((df_immigrantion - df_emigrantion) /5).round().astype(int)
df__proporcao_saldo = (
    df__proporcao_saldo
    .reset_index()
    .groupby(by=['Ano', 'Município', 'Sexo', 'Faixa Etária'], as_index=False)
    .sum()
    .rename(columns={'Peso':'Saldo Migratório'})
#    .pivot(index=['Ano','Município'], columns=['Sexo', 'Faixa Etária'])
)

# Agrupar por Ano, Município e Sexo e calcular a soma total do saldo migratório para cada grupo
df__proporcao_saldo['Total Saldo Migratório por Grupo'] = df__proporcao_saldo.groupby(['Ano', 'Município', 'Sexo'])['Saldo Migratório'].transform('sum')

# Calcular a proporção do saldo migratório para cada linha em relação ao total do grupo
df__proporcao_saldo['Proporção do Saldo Migratório'] = df__proporcao_saldo['Saldo Migratório'] / df__proporcao_saldo['Total Saldo Migratório por Grupo']

df__proporcao_saldo.to_csv('../../br_demography/results/tab/proporcoes_saldo_migratorio.csv', sep=';', decimal=',', encoding='utf-16', index=False)

df__proporcao_saldo.head()

,Ano,Município,Sexo,Faixa Etária,Saldo Migratório,Total Saldo Migratório por Grupo,Proporção do Saldo Migratório
0,2000,Adrianópolis,Feminino,0 a 9 anos,-9,-73,0.123288
1,2000,Adrianópolis,Feminino,10 a 19 anos,-13,-73,0.178082
2,2000,Adrianópolis,Feminino,20 a 29 anos,-31,-73,0.424658
3,2000,Adrianópolis,Feminino,30 a 39 anos,-6,-73,0.082192
4,2000,Adrianópolis,Feminino,40 a 49 anos,-7,-73,0.095890


In [27]:
df_saldo = ((df_immigrantion - df_emigrantion) /5).round().astype(int)
df_saldo = (
    df_saldo
    .reset_index()
    .groupby(by=['Ano', 'Município'], as_index=False)
    .sum()
    .rename(columns={'Peso':'Saldo Migratório'})
    .pivot(index='Município', columns='Ano')
)

df_saldo.head()

Saldo Migratório      
Ano                             2000  2010
Município                                 
Adrianópolis                    -178   -48
Agudos do Sul                    -19    10
Almirante Tamandaré             2469  1380
Araucária                       1893  2302
Balsa Nova                       191   205

In [28]:
df_pop = pd.read_csv(filepath_or_buffer='../../br_demography/results/tab/pop_municipios_rmc_2000_2022_estimativa_intercensitaria.csv', sep=';')
#df_pop[df_pop['Faixa Etária'].isin(values=df_deaths.index)]
df_pop.set_index(keys=['Município', 'Sexo','Faixa Etária'], inplace=True)
df_pop.columns = df_pop.columns.astype(int)
df_pop = (
    df_pop
    .reset_index()
    .groupby(by=['Município'])[2010, 2022]
    .sum()
)

df_pop['Crescimento Pop 2010-2022'] = df_pop[2022] - df_pop[2010]

df_pop.iloc[np.r_[0:2, -2:0]]

,2010,2022,Crescimento Pop 2010-2022
Município,,,
Adrianópolis,6376,6256,-120
Agudos do Sul,8270,10233,1963
Tijucas do Sul,14538,17621,3083
Tunas do Paraná,6256,6219,-37


In [34]:
df_deaths = pd.read_csv('../../br_demography/results/tab/obitos_rmc_2000_2022.csv', sep=';', decimal=',', encoding='utf-16')

df_deaths = (
    df_deaths
    .groupby(by='Município')
    .sum()
    .loc[:,list(str(year) for year in range(2010, 2022))]
    .sum(axis=1)
    .to_frame()
    .rename(columns={0:'Óbitos 2010 a 2022'})
)

In [54]:
df_births = pd.read_csv('../../br_demography/results/tab/nascimentos_rmc_2000_2022.csv', sep=';', decimal=',', encoding='utf-16')
df_births = (
    df_births
    .drop(columns=['Unnamed: 0'])
    .groupby(by='Município')
    .sum()
    .loc[:,list(str(year) for year in range(2010, 2022))]
    .sum(axis=1)
    .to_frame()
    .rename(columns={0:'Nascimentos 2010 a 2022'})
)

In [58]:
df_pop = pd.concat(
    objs=[
        df_pop
        ,df_births
        ,df_deaths
    ]
    , axis=1
)

In [61]:
df_pop['Crescimento Vegetativo 2010 a 2022'] = df_pop['Nascimentos 2010 a 2022'] - df_pop['Óbitos 2010 a 2022']

In [71]:
df_saldo['2022'] = ((df_pop['Crescimento Pop 2010-2022'] - df_pop['Crescimento Vegetativo 2010 a 2022']) / 12).round(0)

In [77]:
df_saldo = df_saldo.astype(int)
df_saldo.to_csv('../../br_demography/results/tab/saldo_migratorio_anual.csv', sep=';', decimal=',', encoding='utf-16')

In [24]:
(
    df_pop
    .reset_index()
    .groupby(by=['Município'])[[2022]]
    .sum()
)

,2022
Município,
Adrianópolis,6256
Agudos do Sul,10233
Almirante Tamandaré,119825
Araucária,151666
Balsa Nova,13395
Bocaiúva do Sul,13299
Campina Grande do Sul,47825
Campo Largo,136327
Campo Magro,30160


## Versão Original

In [11]:
df_emigration_2010 = mm.query_emigration_by_sex_age(mun_id=4125506, project_id=project_id)
df_emigration_2010 = mm.standard_age_groups(df=df_emigration_2010, age_group_csv_path='../../br_demography/source/tab/faixas_etarias_censo_2000_2010.csv')
print(df_emigration_2010.sum())
df_emigration_2010

Downloading: 100%|██████████| 151/151 [00:00<00:00, 472.81rows/s]

Peso    17572.015176
dtype: float64


Peso
Sexo      Faixa Etária                
Feminino  0 a 9 anos        851.104868
          10 a 19 anos     1862.462907
          20 a 29 anos     2104.799079
          30 a 39 anos     2113.455909
          40 a 49 anos      945.200576
          50 a 59 anos      584.163988
          60 a 69 anos      363.652692
          70 a 79 anos      127.786227
          80 anos ou mais    47.482221
Masculino 0 a 9 anos       1031.017653
          10 a 19 anos     1463.050411
          20 a 29 anos     1883.404714
          30 a 39 anos     1996.357163
          40 a 49 anos      994.074042
          50 a 59 anos      756.153361
          60 a 69 anos      319.895357
          70 a 79 anos      100.278330
          80 anos ou mais    27.675679

In [12]:
df_immigration_2010 = mm.query_immigration_by_sex_age(mun_id=4125506, project_id=project_id)
df_immigration_2010 = mm.standard_age_groups(df=df_immigration_2010, age_group_csv_path='../../br_demography/source/tab/faixas_etarias_censo_2000_2010.csv')
print(df_immigration_2010.sum())
df_immigration_2010

Downloading: 100%|██████████| 158/158 [00:00<00:00, 492.89rows/s]

Peso    42278.013782
dtype: float64


Peso
Sexo      Faixa Etária                
Feminino  0 a 9 anos       1848.448294
          10 a 19 anos     3809.449305
          20 a 29 anos     6158.404003
          30 a 39 anos     4692.532438
          40 a 49 anos     2219.688942
          50 a 59 anos     1224.530322
          60 a 69 anos      744.159626
          70 a 79 anos      318.192755
          80 anos ou mais   101.098137
Masculino 0 a 9 anos       2038.369550
          10 a 19 anos     3702.836286
          20 a 29 anos     5494.576097
          30 a 39 anos     5003.829450
          40 a 49 anos     2661.795551
          50 a 59 anos     1289.293916
          60 a 69 anos      615.445002
          70 a 79 anos      308.073700
          80 anos ou mais    47.290408

In [13]:
df_immigration_2000 = mm.query_immigration_by_sex_age_2000(mun_id=4125506, project_id=project_id)
df_immigration_2000 = mm.standard_age_groups(df=df_immigration_2000, age_group_csv_path='../../br_demography/source/tab/faixas_etarias_censo_2000_2010.csv')
print(df_immigration_2000.sum())
df_immigration_2000

Downloading: 100%|██████████| 159/159 [00:00<00:00, 386.82rows/s]

Peso    36720.018503
dtype: float64


Peso
Sexo      Faixa Etária                
Feminino  0 a 9 anos       2106.181942
          10 a 19 anos     3703.296473
          20 a 29 anos     5101.829473
          30 a 39 anos     3721.091898
          40 a 49 anos     1720.064059
          50 a 59 anos      918.181820
          60 a 69 anos      498.665733
          70 a 79 anos      211.292967
          80 anos ou mais    97.828150
Masculino 0 a 9 anos       2392.215589
          10 a 19 anos     3522.854820
          20 a 29 anos     5088.914143
          30 a 39 anos     4041.782185
          40 a 49 anos     1855.237730
          50 a 59 anos      960.510104
          60 a 69 anos      505.752297
          70 a 79 anos      219.535709
          80 anos ou mais    54.783411

In [14]:
df_emigration_2000 = mm.query_emigration_by_sex_age_2000(mun_id=4125506, project_id=project_id)
df_emigration_2000 = mm.standard_age_groups(df=df_emigration_2000, age_group_csv_path='../../br_demography/source/tab/faixas_etarias_censo_2000_2010.csv')
print(df_emigration_2000.sum())
df_emigration_2000

Downloading: 100%|██████████| 146/146 [00:00<00:00, 477.63rows/s]

Peso    9719.221228
dtype: float64


Peso
Sexo      Faixa Etária                
Feminino  0 a 9 anos        656.551694
          10 a 19 anos      896.779877
          20 a 29 anos     1252.137432
          30 a 39 anos      885.292356
          40 a 49 anos      598.921375
          50 a 59 anos      231.535103
          60 a 69 anos      156.398416
          70 a 79 anos       80.572836
          80 anos ou mais    67.800153
Masculino 0 a 9 anos        543.658480
          10 a 19 anos     1035.718608
          20 a 29 anos     1059.938562
          30 a 39 anos      937.061766
          40 a 49 anos      692.452060
          50 a 59 anos      372.790616
          60 a 69 anos      175.739964
          70 a 79 anos       62.491979
          80 anos ou mais    13.379951

In [15]:
df_saldo_2000 = ((df_immigration_2000 - df_emigration_2000) /5).round().astype(int)
print(df_saldo_2000.sum())
df_saldo_2000

Peso    5399
dtype: int64


Peso
Sexo      Faixa Etária         
Feminino  0 a 9 anos        290
          10 a 19 anos      561
          20 a 29 anos      770
          30 a 39 anos      567
          40 a 49 anos      224
          50 a 59 anos      137
          60 a 69 anos       68
          70 a 79 anos       26
          80 anos ou mais     6
Masculino 0 a 9 anos        370
          10 a 19 anos      497
          20 a 29 anos      806
          30 a 39 anos      621
          40 a 49 anos      233
          50 a 59 anos      118
          60 a 69 anos       66
          70 a 79 anos       31
          80 anos ou mais     8

In [17]:
df_saldo_2010 = ((df_immigration_2010 - df_emigration_2010) /5).round().astype(int)
print(df_saldo_2010.sum())
df_saldo_2010

Peso    4941
dtype: int64


Peso
Sexo      Faixa Etária         
Feminino  0 a 9 anos        199
          10 a 19 anos      389
          20 a 29 anos      811
          30 a 39 anos      516
          40 a 49 anos      255
          50 a 59 anos      128
          60 a 69 anos       76
          70 a 79 anos       38
          80 anos ou mais    11
Masculino 0 a 9 anos        201
          10 a 19 anos      448
          20 a 29 anos      722
          30 a 39 anos      601
          40 a 49 anos      334
          50 a 59 anos      107
          60 a 69 anos       59
          70 a 79 anos       42
          80 anos ou mais     4

In [8]:
df_saldo.to_csv(path_or_buf='./br_demography/results/tab/saldo_migratorio_anual_2000_2010_campina_2022_2044.csv', sep=';', decimal=',', encoding='utf-16')